# Steam (Environment - Agent)

In [1]:
# Basic import
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from scipy.stats import invgamma, gamma
from scipy.stats import t as student

## Environment

In [2]:
class Environment:
    """ 
    Contextual Multi-Armed Bandit environment
    """
    def __init__(self, nb_films, nb_users, context_size, seed=None):
        self._nb_games = nb_games
        self._nb_users = nb_users
        self._p = context_size # size of user, size of game
        self._rng = np.random.RandomState(seed)
        self._games = self._rng.uniform(size=(nb_games, context_size))
        self._users = self._rng.uniform(size=(nb_users, context_size))
        self._reward_matrix = np.zeros((nb_users, nb_games))
        for i in range(self._reward_matrix.shape[0]):
            for j in range(self._reward_matrix.shape[1]):
                reward = np.linalg.norm(self._games[j] - self._users[i], ord=2) 
                self._reward_matrix[i, j] = reward
        self._reward_matrix = (self._reward_matrix / np.max(self._reward_matrix) * 4).astype(int) + 1
        self._available_games = np.ones((nb_users, nb_games))

    def step(self):
        """ Play an action """
        user = self._rng.randint(0, self._nb_users)
        available_games = np.where(self._available_games[user] == 1)[0]
        return user, available_games
    
    def update(self, user, game):
        reward = self._reward_matrix[user, game]
        self._available_games[user, game] = 0
        return reward
    
    def reset(self):
        self._users = self._rng.uniform(size=(self._nb_users, self._p))
        self._reward_matrix = np.zeros((nb_users, nb_games))
        for i in range(self._reward_matrix.shape[0]):
            for j in range(self._reward_matrix.shape[1]):
                reward = np.linalg.norm(self._games[j] - self._users[i], ord=2) 
                self._reward_matrix[i, j] = reward
        self._reward_matrix = (self._reward_matrix / np.max(self._reward_matrix) * 4).astype(int) + 1
        self._available_films = np.ones((nb_users, nb_games))
        users = deepcopy(self._users)
        return users

## Agent

In [3]:
class RandomAgent:
    """ 
    Random agent
    """
    def __init__(self, seed = None):
        self._rng = np.random.RandomState(seed)
    
    def act(self, available_games):
        action = self._rng.choice(available_games)
        return action

## Experiment

In [4]:
# Basic parameter
nb_users = 30 #number of users in the context
nb_games = 10 #number of games in the context
context_size = 2 #number of different film categories

In [5]:
# Creating the environment
env = Environment(nb_games,nb_users,context_size,2020)
env.reset() #reset and initilize the environment

array([[0.57654812, 0.17727862],
       [0.9944587 , 0.11737487],
       [0.84084483, 0.01402673],
       [0.87168748, 0.90240675],
       [0.44920447, 0.61818198],
       [0.97897813, 0.39727848],
       [0.44723083, 0.23325998],
       [0.93172938, 0.27280247],
       [0.93232779, 0.4132305 ],
       [0.16067842, 0.46880253],
       [0.26928866, 0.94426857],
       [0.29544093, 0.15155985],
       [0.2725433 , 0.86498109],
       [0.91686141, 0.74978059],
       [0.90022206, 0.02240632],
       [0.04095237, 0.89855301],
       [0.70079315, 0.66934705],
       [0.55284578, 0.72274166],
       [0.85393131, 0.60901857],
       [0.45241441, 0.29665953],
       [0.63061983, 0.50703485],
       [0.37025605, 0.13821335],
       [0.75918439, 0.9589782 ],
       [0.27902813, 0.7308044 ],
       [0.19991699, 0.64376932],
       [0.20287344, 0.05044151],
       [0.37199545, 0.01409271],
       [0.09958179, 0.99934911],
       [0.92577299, 0.81490306],
       [0.22237219, 0.09742508]])

In [6]:
# Creating the agent
agent = RandomAgent(2020)

We run the experiment and generate some historical data.

In [7]:
# Running several trials
nb_iteration = 100 #how many trials
rating_matrix = np.zeros((env._nb_users, env._nb_games))
users = list()
games = list()
ratings = list()
for i in range(nb_iteration):
    user, available_games = env.step()
    choosen_game = agent.act(available_games)
    reward = env.update(user, choosen_game)
    users.append(user)
    games.append(choosen_game)
    ratings.append(reward)
    rating_matrix[user, choosen_game] = reward
    '''
    print("user = {}, recommended_games = {}, choosen_game = {}".format(user,recommended_games,choosen_game))
    print("reward = {}\n".format(reward))
    '''
    
print("rating matrix: \n", str(rating_matrix))

rating matrix: 
 [[3. 0. 0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 4. 2. 4. 4. 0.]
 [0. 0. 0. 0. 0. 0. 0. 3. 0. 0.]
 [1. 3. 0. 0. 0. 0. 0. 3. 0. 0.]
 [3. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 2. 0. 0. 0. 0. 0. 3. 0. 0.]
 [0. 1. 0. 1. 0. 3. 2. 0. 2. 0.]
 [3. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 0. 0. 3. 1. 0. 0. 0. 3. 0.]
 [0. 0. 0. 1. 3. 2. 0. 1. 1. 1.]
 [0. 3. 3. 3. 0. 1. 0. 2. 2. 0.]
 [0. 1. 1. 0. 0. 0. 0. 2. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0.]
 [1. 0. 0. 3. 3. 0. 0. 3. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 4. 0.]
 [0. 0. 3. 0. 0. 0. 0. 0. 0. 3.]
 [0. 0. 0. 0. 2. 0. 0. 0. 2. 0.]
 [0. 0. 2. 0. 3. 0. 0. 1. 0. 0.]
 [0. 2. 0. 3. 0. 3. 0. 2. 0. 0.]
 [0. 1. 1. 0. 2. 0. 0. 2. 2. 2.]
 [0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 2. 0. 2. 0.]
 [1. 0. 4. 0. 3. 3. 3. 2. 3. 0.]
 [0. 0. 0. 2. 0. 0. 3. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
 [0. 2. 0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 2. 0. 0. 0. 3. 0. 2.]
 [4. 3. 3. 0. 5. 1. 0. 2. 2. 3.]
 [1. 3. 3. 0. 0. 3. 0. 0. 3. 4.]
 [4. 0. 0. 0. 0. 0. 3. 0. 

## Deep matrix factorization and Embedding agent

In [8]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.layers import Dot
from tensorflow.keras.models import Model

In [9]:
class RegressionModel(Model):
    def __init__(self, embedding_size, max_user, max_game):
        super().__init__()
        
        self.user_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_user,
                                        input_length=1,
                                        name='user_embedding')
        self.game_embedding = Embedding(output_dim=embedding_size,
                                        input_dim=max_game,
                                        input_length=1,
                                        name='game_embedding')
        
        # The following two layers don't have parameters.
        self.flatten = Flatten()
        self.dot = Dot(axes=1)
        
    def call(self, inputs):
        user_inputs = inputs[0]
        game_inputs = inputs[1]
        
        user_vecs = self.flatten(self.user_embedding(user_inputs))
        game_vecs = self.flatten(self.game_embedding(game_inputs))
        
        y = self.dot([user_vecs, game_vecs])
        return y

model = RegressionModel(64, nb_users, nb_games)
model.compile(optimizer="adam", loss='mae')

In [10]:
users = np.array(users)
games = np.array(games)
ratings = np.array(ratings)

history = model.fit([users, games], ratings,
                    batch_size=64, epochs=100, validation_split=0.1,
                    shuffle=True)

embeddings = model.get_weights()

Train on 90 samples, validate on 10 samples
Epoch 1/100
90/90 [==============================] - 0s 4ms/sample - loss: 2.3448 - val_loss: 2.2003
Epoch 2/100
90/90 [==============================] - 0s 207us/sample - loss: 2.3426 - val_loss: 2.2004
Epoch 3/100
90/90 [==============================] - 0s 153us/sample - loss: 2.3407 - val_loss: 2.2006
Epoch 4/100
90/90 [==============================] - 0s 130us/sample - loss: 2.3389 - val_loss: 2.2007
Epoch 5/100
90/90 [==============================] - 0s 226us/sample - loss: 2.3370 - val_loss: 2.2008
Epoch 6/100
90/90 [==============================] - 0s 183us/sample - loss: 2.3351 - val_loss: 2.2008
Epoch 7/100
90/90 [==============================] - 0s 131us/sample - loss: 2.3332 - val_loss: 2.2008
Epoch 8/100
90/90 [==============================] - 0s 273us/sample - loss: 2.3313 - val_loss: 2.2008
Epoch 9/100
90/90 [==============================] - 0s 149us/sample - loss: 2.3292 - val_loss: 2.2007
Epoch 10/100
90/90 [===========

90/90 [==============================] - 0s 383us/sample - loss: 1.0772 - val_loss: 1.1434
Epoch 80/100
90/90 [==============================] - 0s 224us/sample - loss: 1.0520 - val_loss: 1.1260
Epoch 81/100
90/90 [==============================] - 0s 171us/sample - loss: 1.0243 - val_loss: 1.1087
Epoch 82/100
90/90 [==============================] - 0s 160us/sample - loss: 0.9981 - val_loss: 1.0910
Epoch 83/100
90/90 [==============================] - 0s 180us/sample - loss: 0.9707 - val_loss: 1.0734
Epoch 84/100
90/90 [==============================] - 0s 179us/sample - loss: 0.9451 - val_loss: 1.0553
Epoch 85/100
90/90 [==============================] - 0s 167us/sample - loss: 0.9176 - val_loss: 1.0371
Epoch 86/100
90/90 [==============================] - 0s 171us/sample - loss: 0.8925 - val_loss: 1.0182
Epoch 87/100
90/90 [==============================] - 0s 165us/sample - loss: 0.8670 - val_loss: 0.9996
Epoch 88/100
90/90 [==============================] - 0s 200us/sample - loss:

In [11]:
def cosine(x, y):
    dot = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    cos = dot / (norm_x * norm_y)
    return cos

In [12]:
class EmbeddingAgent:
    """ 
    Embedding Agent
    """
    def __init__(self, user_embeddings, game_embeddings):
        self._game_embeddings = game_embeddings
        self._user_embeddings = user_embeddings
    
    def act(self, user, available_games):
        user_embedding = self._user_embeddings[user]
        dot_products = self._game_embeddings @ user_embedding
        user_embedding_norm = np.linalg.norm(user_embedding)
        all_item_norms = np.linalg.norm(self._game_embeddings, axis=1)
        norm_products = user_embedding_norm * all_item_norms
        sims = dot_products / (norm_products)
        sims = np.argsort(sims)[::-1]
        mask = np.in1d(sims, available_games)
        sims = sims[mask]
        return sims[0]

In [13]:
# Creating the agent
agent = EmbeddingAgent(embeddings[0], embeddings[1])

In [14]:
# Running several trials
nb_iteration = 20 #how many trials
for i in range(nb_iteration):
    user, available_games = env.step()
    choosen_game = agent.act(user, available_games)
    reward = env.update(user, choosen_game)
    rating_matrix[user, choosen_game] = reward
    print("user = {}, available games = {}, choosen_game = {}".format(user,available_games,choosen_game))
    print("reward = {}\n".format(reward))

user = 4, available games = [1 2 3 4 5 6 7 8 9], choosen_game = 4
reward = 3

user = 19, available games = [0 3 5 6], choosen_game = 5
reward = 2

user = 29, available games = [1 2 3 4 5 7 8 9], choosen_game = 4
reward = 3

user = 17, available games = [0 1 3 5 6 8 9], choosen_game = 0
reward = 2

user = 17, available games = [1 3 5 6 8 9], choosen_game = 9
reward = 3

user = 1, available games = [0 1 2 3 4 9], choosen_game = 3
reward = 3

user = 17, available games = [1 3 5 6 8], choosen_game = 5
reward = 2

user = 21, available games = [0 1 2 4 5 7 9], choosen_game = 7
reward = 2

user = 16, available games = [0 1 2 3 5 6 7 9], choosen_game = 5
reward = 2

user = 13, available games = [1 2 5 6 8 9], choosen_game = 5
reward = 3

user = 26, available games = [0 1 2 4 5 6 8], choosen_game = 1
reward = 2

user = 6, available games = [0 2 4 7 9], choosen_game = 7
reward = 2

user = 21, available games = [0 1 2 4 5 9], choosen_game = 5
reward = 3

user = 24, available games = [0 1 2 3 4 5 